<a href="https://colab.research.google.com/github/hursoo/Digital_History/blob/main/dh_gb_socialism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1_데이터 불러오기

## 1) 라이브러리, 구글 마운트

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

## 2) 파일 불러오기

In [ ]:
import pandas as pd

In [ ]:
url = 'https://github.com/hursoo/Digital_History/raw/main/gb_corpus.xlsx'

In [ ]:
df = pd.read_excel(url, index_col = 0)
df

In [ ]:
df[:3].to_csv('imsi.txt')

# 2_논조를 반영하는 특성 추출

## 1) 문서-단어 행렬(dtm) 산출 함수

In [ ]:
def get_dtm(df, col_name, rank_n): # rank_n : 고빈도 단어 n 순위까지
    # 단어 종류 모두 벡터화. 2음절 이상
    tv = TfidfVectorizer(norm=None)
    dtm = tv.fit_transform(df[col_name])

    # df 형태로 표시
    dtm_df = pd.DataFrame(dtm.toarray(), columns=tv.get_feature_names_out(), index=df.index)

    highword_list = dtm_df.sum().sort_values(ascending=False)[:rank_n].index.to_list()
    feature_df = dtm_df[highword_list] # 열 순서는 tfidf값이 높은 것부터 낮은 순으로 정렬
    return feature_df

## 2) 특성 및 특성벡터
- tfidf 고빈도 50위 단어

In [ ]:
dtm50_df = get_dtm(df, 'sent_split', 50)
dtm50_df

#3_시기 구분하기
- 시기 구분 : 논조 변화를 관찰하는 마디

## 1) dtm50 + 구간(grid) = gtm50 산출하기

In [ ]:
def transform_to_gtm(df, grid_col, dtm_df): # 인자 - df, 구간 정보 열, dtm50_df
    # 구간정보만 df로 추출
    grid_df = df[[grid_col]]
    # 구간 정보 결합하고, 구간을 index로 만듦
    temp_dtm = pd.concat([dtm_df, grid_df], axis=1)
    grid_dtm = temp_dtm.set_index(grid_col)
    # 구간별 평균
    gtm = grid_dtm.groupby(grid_col).mean()
    return gtm

In [ ]:
# 함수 실행하여 gtm 변수에 할당
gtm_df = transform_to_gtm(df, 'grid', dtm50_df)
gtm_df

## 2) 구간별 상관계수 산출하기

In [ ]:
def make_corr(gtm):
    tgm = gtm.T
    result = tgm.corr()
    return result

In [ ]:
gg_corr = make_corr(gtm_df)
gg_corr

## 3) 히트맵 시각화로 시기구분 하기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def draw_heatmap(df):
    df = df.copy()
    plt.figure(figsize=(10,4))
    sns.heatmap(data = df, annot=True,
    fmt = '.3f', linewidths=.5, cmap='Blues')

In [ ]:
draw_heatmap(gg_corr)

- 모호한 부분의 시기 구분  
 - 해당 구간이 좌우와 가진 상관계수를 비교  
 - 이 때 비교 범위는 왼쪽(및 오른쪽) 가상 시기에 포함되는 구간들의 상관계수를 평균한 값  
 =>   
    1p : 01hf - 05hf  
    2p : 06hf - 09hf  
    3p : 10hf - 12hf

## 4) 시기별 코퍼스 산출

In [ ]:
prd_1 = ['01hf', '02hf', '03hf', '04hf', '05hf']
prd_2 = ['06hf', '07hf', '08hf', '09hf']
prd_3 = ['10hf', '11hf', '12hf']

In [ ]:
df = df.copy()
df['period'] = '' # df에 새 열('period')을 생성
df.loc[df['grid'].isin(prd_1), 'period'] = '1p'
df.loc[df['grid'].isin(prd_2), 'period'] = '2p'
df.loc[df['grid'].isin(prd_3), 'period'] = '3p'
df

In [ ]:
# 시기구분 정보 반영 검증
df.groupby(['period', 'grid'])['sent_id'].count()

In [ ]:
# 시기별 코퍼스

df_1p = df[df['period'] == '1p']
df_2p = df[df['period'] == '2p']
df_3p = df[df['period'] == '3p']

In [ ]:
print(df_1p.shape)
print(df_2p.shape)
print(df_3p.shape)
print(df_1p.shape[0] + df_2p.shape[0] + df_3p.shape[0])

# 4_시기별 연결망 계수, 지표 산출

## 1) 시기별 코사인유사도 계수

In [ ]:
# 코사인유사도 산출 함수

def get_cossim(dtm_df):
    tdf_n = dtm_df.columns.tolist()
    tdm = dtm_df.T
    cossim = cosine_similarity(tdm, tdm)
    cossim_df = pd.DataFrame(cossim, columns=tdf_n, index=tdf_n)
    return cossim_df

In [ ]:
# 시기별 dtm 산출

dtm50_df_1p = get_dtm(df_1p, 'sent_split', 50)
dtm50_df_2p = get_dtm(df_2p, 'sent_split', 50)
dtm50_df_3p = get_dtm(df_3p, 'sent_split', 50)

In [ ]:
print(dtm50_df_1p.shape)
print(dtm50_df_2p.shape)
print(dtm50_df_3p.shape)

In [ ]:
# 시기별 코사인 유사도 산출 함수 실행

cossim_1p = get_cossim(dtm50_df_1p)
cossim_2p = get_cossim(dtm50_df_2p)
cossim_3p = get_cossim(dtm50_df_3p)

In [ ]:
print(cossim_1p.iloc[:3,:5])
print(cossim_2p.iloc[:3,:5])
print(cossim_3p.iloc[:3,:5])

## 2) wnet.exe 입력값 작성

In [ ]:
# 코사인유사도 df를 그대로 텍스트 파일(tsv)로 저장 (인덱스, 헤더 모두 출력하도록 해야 고빈도 50개 단어가 출력됨)

cossim_1p.to_csv('wnet_input_1p.txt', index=True, header=True, sep='\t')
cossim_2p.to_csv('wnet_input_2p.txt', index=True, header=True, sep='\t')
cossim_3p.to_csv('wnet_input_3p.txt', index=True, header=True, sep='\t')

# The End of Note